In [38]:
import random
import torch
import torch.nn as nn
import torch.multiprocessing as mp
mp.set_start_method('spawn', force=True)
# import data as data
# import model_evaluation as evaluation
import torch.optim as optim
import torch._dynamo
import numpy as np
import heapq
from torch.optim.lr_scheduler import ReduceLROnPlateau

torch._dynamo.config.suppress_errors = True

random.seed(1000)

# (user_id: [(movie_id, label)])

In [39]:
def load_data_rate(filename, threshold=3, train_ratio=0.7, test_ratio=0.15):
    """
    Load dataset and split data on a per-user basis.

    Args:
        filename (str): Path to the ratings file.
        train_ratio (float): Percentage of interactions used for training.
        test_ratio (float): Percentage of interactions used for testing.

    Returns:
        train_dict, val_dict, test_dict, movie_num, user_num
    """
    user_ratings = {}  # Store each user's interactions (user_id: [(movie_id, label), ...])
    movie_num = -1
    user_num = -1

    with open(filename, "r", encoding="utf-8") as file:
        for line in file:
            user_id, movie_id, rating, _ = map(int, line.strip().split("::"))
            label = 1 if rating >= threshold else 0

            if user_id not in user_ratings:
                user_ratings[user_id] = []
            user_ratings[user_id].append((movie_id, label))

            # movie and user number
            movie_num = max(movie_num, movie_id)
            user_num = max(user_num, user_id)

    train_dict, val_dict, test_dict = {}, {}, {}

    ######### divide by users? cold start #######
    # Divide each user's movie interactions by proportion
    for user_id, interactions in user_ratings.items():
        random.shuffle(interactions)  # shuffle

        total_interactions = len(interactions)
        train_end = int(train_ratio * total_interactions)
        val_end = int((train_ratio + test_ratio) * total_interactions)

        train_dict[user_id] = interactions[:train_end]
        val_dict[user_id] = interactions[train_end:val_end]
        test_dict[user_id] = interactions[val_end:]

    return train_dict, val_dict, test_dict, movie_num, user_num

# noninteract movies (for validation and test set to rank)

In [40]:
def get_input_data_nointeract(train_dict, non_interacted_movies):
    user_input, movie_input, labels = [], [], []

    for u, rate_list in train_dict.items():
        # positive samples in train set
        for movie_id, label in rate_list:
            user_input.append(u)
            movie_input.append(movie_id)
            labels.append(label)

        # collect all movies not interacted with user
        non_interacted_items = non_interacted_movies.get(u, [])

        # Add all non-interacted movies as negative samples with label 0
        for movie_id in non_interacted_items:
            user_input.append(u)
            movie_input.append(movie_id)
            labels.append(0)

    return user_input, movie_input, labels

# get input data for training set with negative sampling

In [41]:
######### rate>=4 negative sample? interaction #######
def get_input_data(train_dict, non_interacted_movies, negative_num):
    user_input, movie_input, labels = [], [], []

    for u, rate_list in train_dict.items():
        # positive samples in train set
        for movie_id, label in rate_list:
            user_input.append(u)
            movie_input.append(movie_id)
            labels.append(label)

        # collect all movies not interacted with user
        non_interacted_items = non_interacted_movies.get(u, [])
        # negative samples
        if len(non_interacted_items) >= negative_num:
            negative_samples = random.sample(non_interacted_items, negative_num)
        else:
            negative_samples = list(non_interacted_items) + random.choices(list(non_interacted_items), k=negative_num - len(non_interacted_items))

        for movie_id in negative_samples:
            user_input.append(u)
            movie_input.append(movie_id)
            labels.append(0)

    return user_input, movie_input, labels

In [42]:
def get_non_interacted_movies(train_dict, val_dict, test_dict, movie_num):
    non_interacted_movies = {}

    for u in train_dict:
        # Get the movies that the user has interacted with (including train, val, test)
        interacted_movies = set(movie_id for movie_id, _ in train_dict.get(u, []))
        if u in val_dict:
            interacted_movies.update(movie_id for movie_id, _ in val_dict.get(u, []))
        if u in test_dict:
            interacted_movies.update(movie_id for movie_id, _ in test_dict.get(u, []))

        # Get the movies that the user has not interacted with
        all_movies = set(range(1, movie_num + 1))
        non_interacted_movies[u] = list(all_movies - interacted_movies)

    return non_interacted_movies

# NeuMF

In [43]:
class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, mf_dim=10, layers=[10]):
        super(NeuMF, self).__init__()

        # GMF Embeddings
        self.user_embedding_gmf = nn.Embedding(num_users, mf_dim)
        self.item_embedding_gmf = nn.Embedding(num_items, mf_dim)

        # MLP Embeddings
        self.user_embedding_mlp = nn.Embedding(num_users, layers[0] // 2)
        self.item_embedding_mlp = nn.Embedding(num_items, layers[0] // 2)

        # Initialize embedding weights
        nn.init.normal_(self.user_embedding_gmf.weight, std=0.01)
        nn.init.normal_(self.item_embedding_gmf.weight, std=0.01)
        nn.init.normal_(self.user_embedding_mlp.weight, std=0.01)
        nn.init.normal_(self.item_embedding_mlp.weight, std=0.01)

        # MLP Layers
        self.mlp_layers = nn.Sequential()
        input_dim = layers[0]  # Initial input size (concatenated user & item embeddings)
        for i in range(1, len(layers)):
            self.mlp_layers.add_module(f"fc{i}", nn.Linear(input_dim, layers[i]))
            self.mlp_layers.add_module(f"relu{i}", nn.ReLU())
            input_dim = layers[i]

        # Output layer: combines GMF and MLP outputs
        self.fc_output = nn.Linear(mf_dim + layers[-1], 1)  # GMF (mf_dim) + MLP (last layer size)

    def forward(self, user_indices, item_indices):
        """ Forward pass for NeuMF model """

        # GMF Forward Pass: Element-wise multiplication
        user_latent_gmf = self.user_embedding_gmf(user_indices)
        item_latent_gmf = self.item_embedding_gmf(item_indices)
        gmf_out = torch.mul(user_latent_gmf, item_latent_gmf)  # Element-wise multiplication

        # MLP Forward Pass: Concatenate embeddings and pass through MLP layers
        user_latent_mlp = self.user_embedding_mlp(user_indices)
        item_latent_mlp = self.item_embedding_mlp(item_indices)
        mlp_input = torch.cat((user_latent_mlp, item_latent_mlp), dim=-1)  # Concatenation
        mlp_out = self.mlp_layers(mlp_input)

        # Combine GMF and MLP outputs
        combined = torch.cat((gmf_out, mlp_out), dim=-1)
        prediction = torch.sigmoid(self.fc_output(combined))  # Final prediction

        return prediction

In [44]:
import heapq
from sklearn.metrics import precision_score, recall_score, f1_score

In [45]:
# def model_evaluation(model, val_dict, device, K=10):
#     model.to(device)
#     model.eval()
#     user_input = []
#     movie_input = []
#     labels = []

#     for u, interactions in val_dict.items():
#         for movie_id, label in interactions:
#             user_input.append(u)
#             movie_input.append(movie_id)
#             labels.append(label)

#     user_input = torch.tensor(user_input, dtype=torch.long, device=device)
#     movie_input = torch.tensor(movie_input, dtype=torch.long, device=device)

#     with torch.no_grad():
#         predictions = model(user_input, movie_input).squeeze(-1).cpu().numpy()

#     predictions_dict = {}
#     for u, m, score in zip(user_input.cpu().tolist(), movie_input.cpu().tolist(), predictions):
#         if u not in predictions_dict:
#             predictions_dict[u] = {}
#         predictions_dict[u][m] = score

#     precision_list = []
#     recall_list = []

#     for u, interactions in val_dict.items():
#         pos_movies = {m for m, label in interactions if label == 1}
#         if not pos_movies:
#             continue

#         if u not in predictions_dict:
#             continue
#         pred_scores = predictions_dict[u]

#         top_k_items = np.array(sorted(pred_scores.keys(), key=lambda x: pred_scores[x], reverse=True))[:K]

#         # Calculate Precision@10
#         relevant_in_top_k = sum(1 for movie_id in top_k_items if movie_id in pos_movies)
#         precision_at_10 = relevant_in_top_k / K
#         precision_list.append(precision_at_10)

#         # Calculate Recall@10
#         recall_at_10 = relevant_in_top_k / len(pos_movies)
#         recall_list.append(recall_at_10)

#     # Calculate average Precision@10 and Recall@10
#     avg_precision_at_10 = np.mean(precision_list) if precision_list else 0
#     avg_recall_at_10 = np.mean(recall_list) if recall_list else 0

#     # Calculate F1@10
#     if avg_precision_at_10 + avg_recall_at_10 > 0:
#         f1_at_10 = 2 * (avg_precision_at_10 * avg_recall_at_10) / (avg_precision_at_10 + avg_recall_at_10)
#     else:
#         f1_at_10 = 0

#     return avg_precision_at_10, avg_recall_at_10, f1_at_10

In [46]:
def model_evaluation(model, val_dict, device, K=10):
    model.to(device)
    model.eval()
    user_input = []
    movie_input = []
    labels = []

    for u, interactions in val_dict.items():
        for movie_id, label in interactions:
            user_input.append(u)
            movie_input.append(movie_id)
            labels.append(label)

    user_input = torch.tensor(user_input, dtype=torch.long, device=device)
    movie_input = torch.tensor(movie_input, dtype=torch.long, device=device)

    with torch.no_grad():
        predictions = model(user_input, movie_input).squeeze(-1).cpu().numpy()

    predictions_dict = {}
    for u, m, score in zip(user_input.cpu().tolist(), movie_input.cpu().tolist(), predictions):
        if u not in predictions_dict:
            predictions_dict[u] = {}
        predictions_dict[u][m] = score

    recall_list = []
    ndcg_list = []

    for u, interactions in val_dict.items():
        pos_movies = {m for m, label in interactions if label == 1}
        if not pos_movies:
            continue

        if u not in predictions_dict:
            continue
        pred_scores = predictions_dict[u]

        top_k_items = np.array(sorted(pred_scores.keys(), key=lambda x: pred_scores[x], reverse=True))[:K]

        recall = len(pos_movies.intersection(top_k_items)) / len(pos_movies)
        recall_list.append(recall)

        ndcg = calculate_ndcg(pos_movies, top_k_items, K)
        ndcg_list.append(ndcg)

    avg_recall = np.mean(recall_list) if recall_list else 0
    avg_ndcg = np.mean(ndcg_list) if ndcg_list else 0

    return avg_recall, avg_ndcg


def calculate_ndcg(pos_movies, top_k_items, K):
    """
    Calculate NDCG for the top-K recommended items.

    Args:
    - pos_movies: A set of relevant (ground truth) items for the user.
    - top_k_items: A list of the top-K recommended items.
    - K: The number of top items considered for evaluation.

    Returns:
    - NDCG score.
    """
    K = min(K, len(top_k_items))  # Adjust K to avoid overestimation

    # Compute DCG
    dcg = sum(1 / np.log2(i + 2) for i, item in enumerate(top_k_items[:K]) if item in pos_movies)

    # Compute IDCG (Ideal DCG)
    ideal_hits = min(K, len(pos_movies))  # Can't be more than positive items
    idcg = sum(1 / np.log2(i + 2) for i in range(ideal_hits))

    return dcg / idcg if idcg > 0 else 0

In [47]:
from collections import defaultdict

# NeuMF

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dict, val_dict, test_dict, movie_num, user_num = load_data_rate('/content/ratings.dat', threshold=3)
batch_size = 128
num_epochs = 30

negative_nums = [5]  # Different values of negative samples to test
results = {}  # Dictionary to store results for each negative_num

# Loop through each negative_num
for negative_num in negative_nums:
    # Update non-interacted movies list for the current negative_num
    non_interacted_movies = get_non_interacted_movies(train_dict, val_dict, test_dict, movie_num)

    # Get input data for training with the current negative_num
    user_input, movie_input, labels = get_input_data(train_dict, non_interacted_movies, negative_num)

    # Reinitialize the model, optimizer, and other components
    model_ncf = NeuMF(user_num + 1, movie_num + 1, 10, [10, 16]).to(device)
    model_ncf = torch.compile(model_ncf)
    optimizer = optim.Adam(model_ncf.parameters(), lr=0.0001, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)

    criterion = nn.BCEWithLogitsLoss()  # Loss function
    scaler = torch.amp.GradScaler('cuda')  # Automatic mixed precision scaler

    # Convert input data to tensors and load them to device
    user_input = torch.tensor(user_input, dtype=torch.long).to(device)
    movie_input = torch.tensor(movie_input, dtype=torch.long).to(device)
    labels = torch.tensor(labels, dtype=torch.float32).to(device)

    dataset = torch.utils.data.TensorDataset(user_input, movie_input, labels)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=8)

    # Prepare validation data
    val_user_input, val_movie_input, val_labels = get_input_data_nointeract(val_dict, non_interacted_movies)
    val_dict = defaultdict(list)

    # Organize validation data
    for user_id, movie_id, label in zip(val_user_input, val_movie_input, val_labels):
        val_dict[user_id].append((movie_id, label))

    val_dict = dict(val_dict)

    # Convert validation data to tensors
    val_user_input = torch.tensor(val_user_input, dtype=torch.long).to(device)
    val_movie_input = torch.tensor(val_movie_input, dtype=torch.long).to(device)
    val_labels = torch.tensor(val_labels, dtype=torch.float32).to(device)

    val_dataset = torch.utils.data.TensorDataset(val_user_input, val_movie_input, val_labels)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

    # Start training for the current negative_num
    train_losses_ncf = []  # List to store training losses
    val_losses_ncf = []  # List to store validation losses
    recalls_ncf = []  # List to store recall scores
    ndcgs_ncf = []  # List to store NDCG scores
    patience = 10  # Patience for early stopping
    counter = 0  # Counter for early stopping
    best_val_loss = float('inf')  # Best validation loss

    # Loop through epochs
    for epoch in range(num_epochs):
        model_ncf.train()  # Set model to training mode
        total_loss = 0

        # Train the model with the training data
        for batch_users, batch_items, batch_labels in dataloader:
            batch_users = batch_users.to(device)
            batch_items = batch_items.to(device)
            batch_labels = batch_labels.to(device)
            optimizer.zero_grad(set_to_none=True)
            with torch.amp.autocast('cuda'):
                predictions = model_ncf(batch_users, batch_items)
                loss = criterion(predictions, batch_labels.view(-1, 1))
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()

        print(f"Epoch {epoch + 1}, Training Loss: {total_loss / len(dataloader)}")

        # Validate the model
        model_ncf.eval()  # Set model to evaluation mode
        val_loss = 0
        with torch.no_grad():
            for batch_users, batch_items, batch_labels in val_dataloader:
                batch_users = batch_users.to(device)
                batch_items = batch_items.to(device)
                batch_labels = batch_labels.to(device)
                predictions = model_ncf(batch_users, batch_items)
                loss = criterion(predictions, batch_labels.view(-1, 1))
                val_loss += loss.item()
            val_loss_avg = val_loss / len(val_dataloader)
            scheduler.step(val_loss_avg)  # Adjust learning rate based on validation loss
            print(f"Epoch {epoch + 1}, Validation Loss: {val_loss_avg}")

        # Save losses for later analysis
        train_losses_ncf.append(total_loss / len(dataloader))
        val_losses_ncf.append(val_loss_avg)

        # Evaluate model's performance
        recall, ndcg = model_evaluation(model_ncf, val_dict, device, K=10)
        recalls_ncf.append(recall)
        ndcgs_ncf.append(ndcg)

        # Early stopping check
        if val_loss_avg < best_val_loss:
            best_val_loss = val_loss_avg
            counter = 0
            # Save the best model
            torch.save(model_ncf.state_dict(), f"./best_model_ncf_{negative_num}.pth")
        else:
            counter += 1
            print(f"Early Stopping Counter: {counter}/{patience}")
            if counter >= patience:
                print("Early stopping triggered! Stopping training.")
                break

    # Save the results for the current negative_num
    results[negative_num] = {
        "train_losses": train_losses_ncf,
        "val_losses": val_losses_ncf,
        "recalls": recalls_ncf,
        "ndcgs": ndcgs_ncf
    }

Epoch 1, Training Loss: 0.5259811670086105


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dict, val_dict, test_dict, movie_num, user_num = load_data_rate('/content/ratings.dat', threshold=3)
batch_size = 128
num_epochs = 30

negative_nums = [10]  # Different values of negative samples to test

# Loop through each negative_num
for negative_num in negative_nums:
    # Update non-interacted movies list for the current negative_num
    non_interacted_movies = get_non_interacted_movies(train_dict, val_dict, test_dict, movie_num)

    # Get input data for training with the current negative_num
    user_input, movie_input, labels = get_input_data(train_dict, non_interacted_movies, negative_num)

    # Reinitialize the model, optimizer, and other components
    model_ncf = NeuMF(user_num + 1, movie_num + 1, 10, [10, 16]).to(device)
    model_ncf = torch.compile(model_ncf)
    optimizer = optim.Adam(model_ncf.parameters(), lr=0.0001, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)

    criterion = nn.BCEWithLogitsLoss()  # Loss function
    scaler = torch.amp.GradScaler('cuda')  # Automatic mixed precision scaler

    # Convert input data to tensors and load them to device
    user_input = torch.tensor(user_input, dtype=torch.long).to(device)
    movie_input = torch.tensor(movie_input, dtype=torch.long).to(device)
    labels = torch.tensor(labels, dtype=torch.float32).to(device)

    dataset = torch.utils.data.TensorDataset(user_input, movie_input, labels)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=8)

    # Prepare validation data
    val_user_input, val_movie_input, val_labels = get_input_data_nointeract(val_dict, non_interacted_movies)
    val_dict = defaultdict(list)

    # Organize validation data
    for user_id, movie_id, label in zip(val_user_input, val_movie_input, val_labels):
        val_dict[user_id].append((movie_id, label))

    val_dict = dict(val_dict)

    # Convert validation data to tensors
    val_user_input = torch.tensor(val_user_input, dtype=torch.long).to(device)
    val_movie_input = torch.tensor(val_movie_input, dtype=torch.long).to(device)
    val_labels = torch.tensor(val_labels, dtype=torch.float32).to(device)

    val_dataset = torch.utils.data.TensorDataset(val_user_input, val_movie_input, val_labels)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

    # Start training for the current negative_num
    train_losses_ncf = []  # List to store training losses
    val_losses_ncf = []  # List to store validation losses
    recalls_ncf = []  # List to store recall scores
    ndcgs_ncf = []  # List to store NDCG scores
    patience = 10  # Patience for early stopping
    counter = 0  # Counter for early stopping
    best_val_loss = float('inf')  # Best validation loss

    # Loop through epochs
    for epoch in range(num_epochs):
        model_ncf.train()  # Set model to training mode
        total_loss = 0

        # Train the model with the training data
        for batch_users, batch_items, batch_labels in dataloader:
            batch_users = batch_users.to(device)
            batch_items = batch_items.to(device)
            batch_labels = batch_labels.to(device)
            optimizer.zero_grad(set_to_none=True)
            with torch.amp.autocast('cuda'):
                predictions = model_ncf(batch_users, batch_items)
                loss = criterion(predictions, batch_labels.view(-1, 1))
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()

        print(f"Epoch {epoch + 1}, Training Loss: {total_loss / len(dataloader)}")

        # Validate the model
        model_ncf.eval()  # Set model to evaluation mode
        val_loss = 0
        with torch.no_grad():
            for batch_users, batch_items, batch_labels in val_dataloader:
                batch_users = batch_users.to(device)
                batch_items = batch_items.to(device)
                batch_labels = batch_labels.to(device)
                predictions = model_ncf(batch_users, batch_items)
                loss = criterion(predictions, batch_labels.view(-1, 1))
                val_loss += loss.item()
            val_loss_avg = val_loss / len(val_dataloader)
            scheduler.step(val_loss_avg)  # Adjust learning rate based on validation loss
            print(f"Epoch {epoch + 1}, Validation Loss: {val_loss_avg}")

        # Save losses for later analysis
        train_losses_ncf.append(total_loss / len(dataloader))
        val_losses_ncf.append(val_loss_avg)

        # Evaluate model's performance
        recall, ndcg = model_evaluation(model_ncf, val_dict, device, K=10)
        recalls_ncf.append(recall)
        ndcgs_ncf.append(ndcg)

        # Early stopping check
        if val_loss_avg < best_val_loss:
            best_val_loss = val_loss_avg
            counter = 0
            # Save the best model
            torch.save(model_ncf.state_dict(), f"./best_model_ncf_{negative_num}.pth")
        else:
            counter += 1
            print(f"Early Stopping Counter: {counter}/{patience}")
            if counter >= patience:
                print("Early stopping triggered! Stopping training.")
                break

    # Save the results for the current negative_num
    results[negative_num] = {
        "train_losses": train_losses_ncf,
        "val_losses": val_losses_ncf,
        "recalls": recalls_ncf,
        "ndcgs": ndcgs_ncf
    }

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dict, val_dict, test_dict, movie_num, user_num = load_data_rate('/content/ratings.dat', threshold=3)
batch_size = 128
num_epochs = 30

negative_nums = [15]  # Different values of negative samples to test

# Loop through each negative_num
for negative_num in negative_nums:
    # Update non-interacted movies list for the current negative_num
    non_interacted_movies = get_non_interacted_movies(train_dict, val_dict, test_dict, movie_num)

    # Get input data for training with the current negative_num
    user_input, movie_input, labels = get_input_data(train_dict, non_interacted_movies, negative_num)

    # Reinitialize the model, optimizer, and other components
    model_ncf = NeuMF(user_num + 1, movie_num + 1, 10, [10, 16]).to(device)
    model_ncf = torch.compile(model_ncf)
    optimizer = optim.Adam(model_ncf.parameters(), lr=0.0001, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)

    criterion = nn.BCEWithLogitsLoss()  # Loss function
    scaler = torch.amp.GradScaler('cuda')  # Automatic mixed precision scaler

    # Convert input data to tensors and load them to device
    user_input = torch.tensor(user_input, dtype=torch.long).to(device)
    movie_input = torch.tensor(movie_input, dtype=torch.long).to(device)
    labels = torch.tensor(labels, dtype=torch.float32).to(device)

    dataset = torch.utils.data.TensorDataset(user_input, movie_input, labels)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=8)

    # Prepare validation data
    val_user_input, val_movie_input, val_labels = get_input_data_nointeract(val_dict, non_interacted_movies)
    val_dict = defaultdict(list)

    # Organize validation data
    for user_id, movie_id, label in zip(val_user_input, val_movie_input, val_labels):
        val_dict[user_id].append((movie_id, label))

    val_dict = dict(val_dict)

    # Convert validation data to tensors
    val_user_input = torch.tensor(val_user_input, dtype=torch.long).to(device)
    val_movie_input = torch.tensor(val_movie_input, dtype=torch.long).to(device)
    val_labels = torch.tensor(val_labels, dtype=torch.float32).to(device)

    val_dataset = torch.utils.data.TensorDataset(val_user_input, val_movie_input, val_labels)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

    # Start training for the current negative_num
    train_losses_ncf = []  # List to store training losses
    val_losses_ncf = []  # List to store validation losses
    recalls_ncf = []  # List to store recall scores
    ndcgs_ncf = []  # List to store NDCG scores
    patience = 10  # Patience for early stopping
    counter = 0  # Counter for early stopping
    best_val_loss = float('inf')  # Best validation loss

    # Loop through epochs
    for epoch in range(num_epochs):
        model_ncf.train()  # Set model to training mode
        total_loss = 0

        # Train the model with the training data
        for batch_users, batch_items, batch_labels in dataloader:
            batch_users = batch_users.to(device)
            batch_items = batch_items.to(device)
            batch_labels = batch_labels.to(device)
            optimizer.zero_grad(set_to_none=True)
            with torch.amp.autocast('cuda'):
                predictions = model_ncf(batch_users, batch_items)
                loss = criterion(predictions, batch_labels.view(-1, 1))
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()

        print(f"Epoch {epoch + 1}, Training Loss: {total_loss / len(dataloader)}")

        # Validate the model
        model_ncf.eval()  # Set model to evaluation mode
        val_loss = 0
        with torch.no_grad():
            for batch_users, batch_items, batch_labels in val_dataloader:
                batch_users = batch_users.to(device)
                batch_items = batch_items.to(device)
                batch_labels = batch_labels.to(device)
                predictions = model_ncf(batch_users, batch_items)
                loss = criterion(predictions, batch_labels.view(-1, 1))
                val_loss += loss.item()
            val_loss_avg = val_loss / len(val_dataloader)
            scheduler.step(val_loss_avg)  # Adjust learning rate based on validation loss
            print(f"Epoch {epoch + 1}, Validation Loss: {val_loss_avg}")

        # Save losses for later analysis
        train_losses_ncf.append(total_loss / len(dataloader))
        val_losses_ncf.append(val_loss_avg)

        # Evaluate model's performance
        recall, ndcg = model_evaluation(model_ncf, val_dict, device, K=10)
        recalls_ncf.append(recall)
        ndcgs_ncf.append(ndcg)

        # Early stopping check
        if val_loss_avg < best_val_loss:
            best_val_loss = val_loss_avg
            counter = 0
            # Save the best model
            torch.save(model_ncf.state_dict(), f"./best_model_ncf_{negative_num}.pth")
        else:
            counter += 1
            print(f"Early Stopping Counter: {counter}/{patience}")
            if counter >= patience:
                print("Early stopping triggered! Stopping training.")
                break

    # Save the results for the current negative_num
    results[negative_num] = {
        "train_losses": train_losses_ncf,
        "val_losses": val_losses_ncf,
        "recalls": recalls_ncf,
        "ndcgs": ndcgs_ncf
    }

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dict, val_dict, test_dict, movie_num, user_num = load_data_rate('/content/ratings.dat', threshold=3)
batch_size = 128
num_epochs = 30

negative_nums = [20]  # Different values of negative samples to test

# Loop through each negative_num
for negative_num in negative_nums:
    # Update non-interacted movies list for the current negative_num
    non_interacted_movies = get_non_interacted_movies(train_dict, val_dict, test_dict, movie_num)

    # Get input data for training with the current negative_num
    user_input, movie_input, labels = get_input_data(train_dict, non_interacted_movies, negative_num)

    # Reinitialize the model, optimizer, and other components
    model_ncf = NeuMF(user_num + 1, movie_num + 1, 10, [10, 16]).to(device)
    model_ncf = torch.compile(model_ncf)
    optimizer = optim.Adam(model_ncf.parameters(), lr=0.0001, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)

    criterion = nn.BCEWithLogitsLoss()  # Loss function
    scaler = torch.amp.GradScaler('cuda')  # Automatic mixed precision scaler

    # Convert input data to tensors and load them to device
    user_input = torch.tensor(user_input, dtype=torch.long).to(device)
    movie_input = torch.tensor(movie_input, dtype=torch.long).to(device)
    labels = torch.tensor(labels, dtype=torch.float32).to(device)

    dataset = torch.utils.data.TensorDataset(user_input, movie_input, labels)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=8)

    # Prepare validation data
    val_user_input, val_movie_input, val_labels = get_input_data_nointeract(val_dict, non_interacted_movies)
    val_dict = defaultdict(list)

    # Organize validation data
    for user_id, movie_id, label in zip(val_user_input, val_movie_input, val_labels):
        val_dict[user_id].append((movie_id, label))

    val_dict = dict(val_dict)

    # Convert validation data to tensors
    val_user_input = torch.tensor(val_user_input, dtype=torch.long).to(device)
    val_movie_input = torch.tensor(val_movie_input, dtype=torch.long).to(device)
    val_labels = torch.tensor(val_labels, dtype=torch.float32).to(device)

    val_dataset = torch.utils.data.TensorDataset(val_user_input, val_movie_input, val_labels)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

    # Start training for the current negative_num
    train_losses_ncf = []  # List to store training losses
    val_losses_ncf = []  # List to store validation losses
    recalls_ncf = []  # List to store recall scores
    ndcgs_ncf = []  # List to store NDCG scores
    patience = 10  # Patience for early stopping
    counter = 0  # Counter for early stopping
    best_val_loss = float('inf')  # Best validation loss

    # Loop through epochs
    for epoch in range(num_epochs):
        model_ncf.train()  # Set model to training mode
        total_loss = 0

        # Train the model with the training data
        for batch_users, batch_items, batch_labels in dataloader:
            batch_users = batch_users.to(device)
            batch_items = batch_items.to(device)
            batch_labels = batch_labels.to(device)
            optimizer.zero_grad(set_to_none=True)
            with torch.amp.autocast('cuda'):
                predictions = model_ncf(batch_users, batch_items)
                loss = criterion(predictions, batch_labels.view(-1, 1))
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()

        print(f"Epoch {epoch + 1}, Training Loss: {total_loss / len(dataloader)}")

        # Validate the model
        model_ncf.eval()  # Set model to evaluation mode
        val_loss = 0
        with torch.no_grad():
            for batch_users, batch_items, batch_labels in val_dataloader:
                batch_users = batch_users.to(device)
                batch_items = batch_items.to(device)
                batch_labels = batch_labels.to(device)
                predictions = model_ncf(batch_users, batch_items)
                loss = criterion(predictions, batch_labels.view(-1, 1))
                val_loss += loss.item()
            val_loss_avg = val_loss / len(val_dataloader)
            scheduler.step(val_loss_avg)  # Adjust learning rate based on validation loss
            print(f"Epoch {epoch + 1}, Validation Loss: {val_loss_avg}")

        # Save losses for later analysis
        train_losses_ncf.append(total_loss / len(dataloader))
        val_losses_ncf.append(val_loss_avg)

        # Evaluate model's performance
        recall, ndcg = model_evaluation(model_ncf, val_dict, device, K=10)
        recalls_ncf.append(recall)
        ndcgs_ncf.append(ndcg)

        # Early stopping check
        if val_loss_avg < best_val_loss:
            best_val_loss = val_loss_avg
            counter = 0
            # Save the best model
            torch.save(model_ncf.state_dict(), f"./best_model_ncf_{negative_num}.pth")
        else:
            counter += 1
            print(f"Early Stopping Counter: {counter}/{patience}")
            if counter >= patience:
                print("Early stopping triggered! Stopping training.")
                break

    # Save the results for the current negative_num
    results[negative_num] = {
        "train_losses": train_losses_ncf,
        "val_losses": val_losses_ncf,
        "recalls": recalls_ncf,
        "ndcgs": ndcgs_ncf
    }

In [ ]:
print(results.keys())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

epochs = np.arange(1, num_epochs + 1)
negative_nums = [5, 10, 15, 20]

plt.figure(figsize=(10, 6))

for negative_num in negative_nums:
    train_losses = results[negative_num]["train_losses"]
    train_losses = np.concatenate([train_losses, [np.nan] * (len(epochs) - len(train_losses))]) if len(train_losses) < len(epochs) else train_losses
    plt.plot(epochs, train_losses, linestyle='--', linewidth=2, label=f'Train Loss (Negative Num: {negative_num})')

plt.title('Training Loss Across Different Negative Samples')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

plt.figure(figsize=(10, 6))

for negative_num in negative_nums:
    val_losses = results[negative_num]["val_losses"]
    val_losses = np.concatenate([val_losses, [np.nan] * (len(epochs) - len(val_losses))]) if len(val_losses) < len(epochs) else val_losses
    plt.plot(epochs, val_losses, linestyle='-.', linewidth=2, label=f'Validation Loss (Negative Num: {negative_num})')

plt.title('Validation Loss Across Different Negative Samples')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

epochs = np.arange(1, num_epochs + 1)
negative_nums = [5, 10, 15, 20]

plt.figure(figsize=(10, 6))

for negative_num in negative_nums:
    recalls = results[negative_num]["recalls"]
    recalls = np.concatenate([recalls, [np.nan] * (len(epochs) - len(recalls))]) if len(recalls) < len(epochs) else recalls
    plt.plot(epochs, recalls, linestyle='-', linewidth=2, label=f'Recall (Negative Num: {negative_num})')

plt.title('Recall Across Different Negative Samples')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

plt.figure(figsize=(10, 6))

for negative_num in negative_nums:
    ndcgs = results[negative_num]["ndcgs"]
    ndcgs = np.concatenate([ndcgs, [np.nan] * (len(epochs) - len(ndcgs))]) if len(ndcgs) < len(epochs) else ndcgs
    plt.plot(epochs, ndcgs, linestyle='-', linewidth=2, label=f'NDCG (Negative Num: {negative_num})')

plt.title('NDCG Across Different Negative Samples')
plt.xlabel('Epoch')
plt.ylabel('NDCG')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# Assuming the device is set and model architectures are defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the NeuMF model (replace with your actual architecture)
model_ncf = NeuMF(user_num+1, movie_num+1, 10, [10, 16]).to(device)

# Define the negative_num or other variable to load the model
negative_nums = [20]  # Example negative numbers, change as needed

# Prepare storage for recall and NDCG scores
recall_scores = []
ndcg_scores = []

# Loop through each negative_num to load and evaluate models
for negative_num in negative_nums:
    # Load the model using the respective file name
    model_ncf.load_state_dict(torch.load(f"./best_model_ncf_{negative_num}.pth"))

    # Set model to evaluation mode
    model_ncf.eval()

    # Prepare the test data
    test_user_input, test_movie_input, test_labels = get_input_data_nointeract(test_dict, non_interacted_movies)
    test_dict = defaultdict(list)

    for user_id, movie_id, label in zip(test_user_input, test_movie_input, test_labels):
        test_dict[user_id].append((movie_id, label))

    test_dict = dict(test_dict)

    # Evaluate the model on the test data
    recall_ncf_test, ndcg_ncf_test = model_evaluation(model_ncf, test_dict, device, K=10)

    # Append the results to the lists
    recall_scores.append(recall_ncf_test)
    ndcg_scores.append(ndcg_ncf_test)

# Models (Negative numbers corresponding to the models)
models = [f'NCF (neg_num={num})' for num in negative_nums]

# Create a bar plot comparing Recall and NDCG scores
x = np.arange(len(models))
width = 0.4

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 5))

# Plot Recall
ax1.bar(x - width/2, recall_scores, width, color='blue')
ax1.set_ylabel('Recall@10', fontsize=12, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(models, fontsize=11)
ax1.set_title('Recall Comparison', fontsize=14, fontweight='bold')
ax1.grid(axis='y', linestyle='--', alpha=0.6)

# Plot NDCG
ax2.bar(x + width/2, ndcg_scores, width, color='green')
ax2.set_ylabel('NDCG@10', fontsize=12, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(models, fontsize=11)
ax2.set_title('NDCG Comparison', fontsize=14, fontweight='bold')
ax2.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

In [ ]:
# print(recall_scores)
# print(ndcg_scores)